In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
dataframe = pd.read_csv('../../Data/data_fix_temp.txt')
columns_to_drop=[
    't_obs', 
    'lokasi',
    'Date', 
    'con_prec.mm.', 
    'mic_prec.mm.', 
    'sha_prec.mm.',
    'lcloud...',	
    'mcloud...', 
    'hcloud...', 
    'clmix.kg.kg.', 
    'wamix.kg.kg.', 
    'hari', 
    'UTC', 
    'LAT', 
    'LON',
    ]

features =  dataframe.drop(columns=columns_to_drop) 
target = dataframe['t_obs']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, shuffle=False)

In [8]:
tf.random.set_seed(42)  #first we set random seed
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(1)
])
model.compile(loss = tf.keras.losses.mae, #mae stands for mean absolute error
              optimizer = tf.keras.optimizers.Adam(), #stochastic GD
              metrics = ['mae'])
model.fit( X_train, y_train, epochs = 10)

Epoch 1/10
13607/13607 [==============================] - 24s 2ms/step - loss: 197.7583 - mae: 197.7583
Epoch 2/10
13607/13607 [==============================] - 22s 2ms/step - loss: 7.9161 - mae: 7.9161
Epoch 3/10
13607/13607 [==============================] - 22s 2ms/step - loss: 8.7656 - mae: 8.7656
Epoch 4/10
13607/13607 [==============================] - 24s 2ms/step - loss: 8.7561 - mae: 8.7561
Epoch 5/10
13607/13607 [==============================] - 26s 2ms/step - loss: 9.0939 - mae: 9.0939
Epoch 6/10
13607/13607 [==============================] - 24s 2ms/step - loss: 8.9979 - mae: 8.9979
Epoch 7/10
13607/13607 [==============================] - 23s 2ms/step - loss: 8.8395 - mae: 8.8395
Epoch 8/10
13607/13607 [==============================] - 22s 2ms/step - loss: 9.1637 - mae: 9.1637
Epoch 9/10
13607/13607 [==============================] - 22s 2ms/step - loss: 8.8207 - mae: 8.8207
Epoch 10/10
13607/13607 [==============================] - 22s 2ms/step - loss: 8.7735 - mae: 8.

In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

test_predictions = model.predict(X_test)

# Evaluate the model on the test set
mse = mean_squared_error(y_test, test_predictions)
rmse = mean_squared_error(y_test, test_predictions, squared=False)
mae = mean_absolute_error(y_test, test_predictions)
r2 = r2_score(y_test, test_predictions)

print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absolute Error: {mae}')
print(f'R2 Score: {r2}\n')

3402/3402 [==============================] - 2s 727us/step
Mean Squared Error: 227.5395656529174
Root Mean Squared Error: 15.084414660599775
Mean Absolute Error: 14.790522602283328
R2 Score: -21.30019006281915



In [14]:
inp_nwp = pd.read_csv('../../Data/MONAS-input_nwp_compile.csv')

test = inp_nwp.drop(columns=['Date', 'LAT', 'LON', 'prec_nwp'])
test = test.rename(
    columns={
        'suhu2m(degC)' : 'suhu2m.degC.',
        'dew2m(degC)' : 'dew2m.degC.',
        'rh2m(%)' : 'rh2m...',
        'wspeed(m/s)' : 'wspeed.m.s.',
        'wdir(deg)' : 'wdir.deg.',
        'lcloud(%)' : 'lcloud...',
        'mcloud(%)' : 'mcloud...' ,
        'hcloud(%)' : 'hcloud...',
        'surpre(Pa)' : 'surpre.Pa.' ,
        'clmix(kg/kg)' : 'clmix.kg.kg.' ,
        'wamix(kg/kg)' : 'wamix.kg.kg.' ,
        'outlr(W/m2)' : 'outlr.W.m2.' ,
        'pblh(m)' : 'pblh.m.',
        'lifcl(m)' : 'lifcl.m.' ,
        'cape(j/kg)' : 'cape.j.kg.' ,
        'mdbz' : 'mdbz' ,
        't950(degC)' : 't950.degC.' ,
        'rh950(%)' : 'rh950...',
        'ws950(m/s)' : 'ws950.m.s.' ,
        'wd950(deg)' : 'wd950.deg.' ,
        't800(degC)' : 't800.degC.' ,
        'rh800(%)' : 'rh800...' ,
        'ws800(m/s)' : 'ws800.m.s.',
        'wd800(deg)' : 'wd800.deg.' ,
        't500(degC)' : 't500.degC.' ,
        'rh500(%)' : 'rh500...' ,
        'ws500(m/s)' : 'ws500.m.s.' ,
        'wd500(deg)' : 'wd500.deg.',
})
#hcloud..., wamix.kg.kg., lcloud..., lokasi, clmix.kg.kg., mcloud...
test = test.drop(columns=['lcloud...','mcloud...', 'hcloud...', 'clmix.kg.kg.', 'wamix.kg.kg.',])

y_val_pred = model.predict(X_test)

mse_val = mean_squared_error(y_test, y_val_pred)
rmse_val = mean_squared_error(y_test, y_val_pred, squared=False)
mae_val = mean_absolute_error(y_test, y_val_pred)
r2_val = r2_score(y_test, y_val_pred)

print('Tanpa Scaling')
print(f'Mean Squared Error: {mse_val}')
print(f'Root Mean Squared Error: {rmse_val}')
print(f'Mean Absolute Error: {mae_val}')
print(f'R2 Score: {r2_val}\n')

3402/3402 [==============================] - 3s 750us/step
Tanpa Scaling
Mean Squared Error: 227.5395656529174
Root Mean Squared Error: 15.084414660599775
Mean Absolute Error: 14.790522602283328
R2 Score: -21.30019006281915



In [15]:
#OUTPUT

combined = pd.concat([dataframe[['Date', 'lokasi', 'suhu2m.degC.', 'LON','LAT','ELEV']],y_test, pd.Series(y_val_pred, index = X_test.index)], axis=1)
combined.columns = ['Date', 'lokasi', 'suhu2m.degC.', 'LON','LAT','ELEV', 't_obs', 'prediction']
combined = combined.dropna()
combined.tail(15)

ValueError: Data must be 1-dimensional, got ndarray of shape (108855, 1) instead